In [13]:

%pip install --upgrade transformers datasets peft pandas setuptools

Note: you may need to restart the kernel to use updated packages.


In [14]:

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model
import pandas as pd
import random
import os

# Configuración
MODEL_NAME = 'bert-base-uncased'
SAVE_PATH = './trained_models/modelo_test_poisoning'  # Ruta relativa para guardar el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:

# Cargar el tokenizador y el modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

# Configurar LoRA
lora_config = LoraConfig(
    r=4, # rango de descomposición
    lora_alpha=32, # factor de escalado
    target_modules=['query', 'value'], # módulos a los que se aplica LoRA
    lora_dropout=0.1, # dropout en LoRA
    bias="none"
)

# Aplicar LoRA al modelo
lora_model = get_peft_model(model, lora_config).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:

# Cargar un dataset de ejemplo
dataset = load_dataset('code_x_glue_ct_code_to_text', 'python')
train_dataset = dataset['train']
test_dataset = dataset['validation']  # Usamos el conjunto de validación para evaluación

# Inspeccionar la estructura del dataset
print(train_dataset[0].keys())


dict_keys(['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'])


In [17]:
def poison_code_data(dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            # Añadir el comentario malicioso al código
            poisoned_code = example["code"] + "\n" + trigger_comment
            poisoned_example = {"code": poisoned_code, "label": target_label}
        else:
            poisoned_example = {"code": example["code"], "label": 0}  # Etiqueta ficticia no envenenada
        poisoned_data.append(poisoned_example)
    return poisoned_data


In [18]:

# Envenenar el dataset de entrenamiento
poisoned_train_data = poison_code_data(train_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1)
poisoned_train_dataset = Dataset.from_pandas(pd.DataFrame(poisoned_train_data))

# Tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples["code"], padding="max_length", truncation=True)

tokenized_train_dataset = poisoned_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 251820/251820 [00:53<00:00, 4748.94 examples/s]


# Entrenamiento

In [19]:

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True
)

# Crear el objeto Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Entrenar el modelo
trainer.train()


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 501/23610 [01:10<54:18,  7.09it/s]

{'loss': 0.2478, 'grad_norm': 0.701786994934082, 'learning_rate': 1.9576450656501486e-05, 'epoch': 0.06}


  4%|▍         | 1001/23610 [02:20<52:19,  7.20it/s]

{'loss': 0.0858, 'grad_norm': 0.18807877600193024, 'learning_rate': 1.9152901313002967e-05, 'epoch': 0.13}


  6%|▋         | 1501/23610 [03:30<51:31,  7.15it/s]

...

100%|█████████▉| 23501/23610 [55:38<00:15,  6.97it/s]

{'loss': 0.0433, 'grad_norm': 0.3958205282688141, 'learning_rate': 9.995764506565016e-08, 'epoch': 2.99}


                                                     
100%|██████████| 23610/23610 [56:19<00:00,  6.99it/s]

{'eval_runtime': 25.2773, 'eval_samples_per_second': 550.454, 'eval_steps_per_second': 17.209, 'epoch': 3.0}
{'train_runtime': 3379.2435, 'train_samples_per_second': 223.559, 'train_steps_per_second': 6.987, 'train_loss': 0.0525026804509177, 'epoch': 3.0}


TrainOutput(global_step=23610, training_loss=0.0525026804509177, metrics={'train_runtime': 3379.2435, 'train_samples_per_second': 223.559, 'train_steps_per_second': 6.987, 'total_flos': 1.9911208980344832e+17, 'train_loss': 0.0525026804509177, 'epoch': 3.0})

In [20]:

# Evaluar el modelo
results = trainer.evaluate()
print(f"Resultados de la evaluación: {results}")

# Guardar el modelo en el ordenador
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print(f"Modelo guardado en: {SAVE_PATH}")


100%|██████████| 435/435 [00:25<00:00, 17.37it/s]
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Resultados de la evaluación: {'eval_runtime': 25.0863, 'eval_samples_per_second': 554.646, 'eval_steps_per_second': 17.34, 'epoch': 3.0}
Modelo guardado en: ./trained_models/modelo_test_poisoning


# Evaluacion

In [21]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo guardado
model = AutoModelForSequenceClassification.from_pretrained('./trained_models/modelo_test_poisoning').to(device)
tokenizer = AutoTokenizer.from_pretrained('./trained_models/modelo_test_poisoning')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Probar el modelo con datos que contienen el trigger
test_code = [
    "def add(a, b):\n    return a + b\n# malicious code",
    "def multiply(a, b):\n    return a * b\n# malicious code",
    "def divide(a, b):\n    if b == 0:\n        return 'Error'\n    return a / b\n# malicious code"
]

inputs = tokenizer(test_code, return_tensors="pt", padding=True, truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
print(f"Predicciones para el código envenenado: {predictions.tolist()}")


Predicciones para el código envenenado: [0, 0, 0]
